In [1]:
import pandas as pd
import wandb
api = wandb.Api()

groups = [
#"CNN_final_exponential",
#"CNN_final_raw",
#"CNN_only_relative_xy",
#"CNN_extra_features_no_leak_xy_free",
#"CNN_extra_features_no_leaking",
"CNN_final_meters",
]

model_dict:dict = {}

# Project is specified by <entity/project-name>
for group in groups:
    runs = api.runs(
    "santi-mbenites-university-of-lisbon/wifi-rssi-gradient-search",
    filters={"group":group}
    )
    for run in runs:
        model_dict[group] = model_dict.get(group,[])
        model_dict[group].append(run.name)


In [2]:
display(model_dict)

{'CNN_final_meters': ['outdoor_and_indoor_run0_depth4_model4',
  'outdoor_and_indoor_run1_depth1_model6',
  'outdoor_and_indoor_run2_depth4_model6',
  'outdoor_and_garage_run0_depth4_model7',
  'outdoor_and_garage_run1_depth4_model0',
  'outdoor_and_garage_run2_depth4_model1',
  'garage_and_indoor_run0_depth2_model5',
  'garage_and_indoor_run1_depth4_model5',
  'garage_and_indoor_run2_depth3_model1',
  'outdoor_indoor_and_garage_run0_depth3_model6',
  'outdoor_indoor_and_garage_run1_depth1_model1',
  'outdoor_indoor_and_garage_run2_depth2_model1']}

In [6]:
import os
from model_evaluation import evaluate_model_on_collections
from consts import (
    group_data, BATCHSIZE, all_collections
)
from feature_lists import DATASET_TO_FEATURE

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

results = {
    #"CNN_final_exponential" : [],
    #"CNN_final_raw" : [],
    #"CNN_only_relative_xy" : [],
    #"CNN_extra_features_no_leak_xy_free" : [],
    "CNN_final_meters" : [],
}
for current_group,all_runs in model_dict.items():

    for current_run in all_runs:
        current_group_data      = group_data[current_group]
        current_model_storage   = current_group_data["model_storage"]
        current_database        = current_group_data["database"]
        current_model_path      = f"{current_model_storage}/{current_run}.pt"
        feature_list            = DATASET_TO_FEATURE[current_database]
        relative_coeficient     = current_group_data["relative_coeficient"]
        model_output_name       = f"{current_group}_{current_run}" 

        if not os.path.exists(current_model_path):
            print(f"Model doesnt exist {model_output_name} with path {current_model_path}")
            continue
        
        try:
            mae, mse, rmse, n = evaluate_model_on_collections(
                current_model_path, all_collections, current_database, BATCHSIZE, None, feature_list
            )
        except Exception as e:
            print(f"Failed Processing {model_output_name} with path {current_model_path}")
            print(e)
            continue
        mae_final = mae * relative_coeficient
        mse_final = mse * relative_coeficient
        rmse_final = rmse * relative_coeficient
        print(f"{model_output_name} | N={n} | MAE={mae_final:.6f} | MSE={mse_final:.6f} | RMSE={rmse_final:.6f}")
        results[current_group].append((model_output_name, mae_final, mse_final, rmse_final))


CNN_final_meters_outdoor_and_indoor_run0_depth4_model4 | N=369936 | MAE=6.346202 | MSE=69.052353 | RMSE=8.309775
CNN_final_meters_outdoor_and_indoor_run1_depth1_model6 | N=369936 | MAE=6.302870 | MSE=68.482317 | RMSE=8.275404
CNN_final_meters_outdoor_and_indoor_run2_depth4_model6 | N=369936 | MAE=6.221397 | MSE=67.807631 | RMSE=8.234539
CNN_final_meters_outdoor_and_garage_run0_depth4_model7 | N=369936 | MAE=6.343304 | MSE=63.899971 | RMSE=7.993746
CNN_final_meters_outdoor_and_garage_run1_depth4_model0 | N=369936 | MAE=6.180169 | MSE=61.913250 | RMSE=7.868497
CNN_final_meters_outdoor_and_garage_run2_depth4_model1 | N=369936 | MAE=6.185483 | MSE=61.777627 | RMSE=7.859875
CNN_final_meters_garage_and_indoor_run0_depth2_model5 | N=369936 | MAE=7.450063 | MSE=89.482044 | RMSE=9.459495
CNN_final_meters_garage_and_indoor_run1_depth4_model5 | N=369936 | MAE=7.269621 | MSE=88.476271 | RMSE=9.406183
CNN_final_meters_garage_and_indoor_run2_depth3_model1 | N=369936 | MAE=8.010736 | MSE=88.007724 | 

In [4]:
import pandas as pd
for group, rows in results.items():

    df = pd.DataFrame(rows, columns=["model", "mae", "mse", "rmse"])
    print(f"GROUP : {group}")
    styled = df.style.background_gradient(cmap="RdYlGn_r")  # _r reverses so green=low, red=high
    display(styled)

GROUP : CNN_final_exponential


,model,mae,mse,rmse


GROUP : CNN_final_raw


,model,mae,mse,rmse


GROUP : CNN_only_relative_xy


,model,mae,mse,rmse


GROUP : CNN_extra_features_no_leak_xy_free


,model,mae,mse,rmse


GROUP : CNN_final_meters


,model,mae,mse,rmse
0,CNN_final_meters_outdoor_and_indoor_run0_depth4_model4,6.346202,69.052353,8.309775
1,CNN_final_meters_outdoor_and_indoor_run1_depth1_model6,6.302870,68.482317,8.275404
2,CNN_final_meters_outdoor_and_indoor_run2_depth4_model6,6.221397,67.807631,8.234539
3,CNN_final_meters_outdoor_and_garage_run0_depth4_model7,6.343304,63.899971,7.993746
4,CNN_final_meters_outdoor_and_garage_run1_depth4_model0,6.180169,61.913250,7.868497
5,CNN_final_meters_outdoor_and_garage_run2_depth4_model1,6.185483,61.777627,7.859875
6,CNN_final_meters_garage_and_indoor_run0_depth2_model5,7.450063,89.482044,9.459495
7,CNN_final_meters_garage_and_indoor_run1_depth4_model5,7.269621,88.476271,9.406183
8,CNN_final_meters_garage_and_indoor_run2_depth3_model1,8.010736,88.007724,9.381243
9,CNN_final_meters_outdoor_indoor_and_garage_run0_depth3_model6,5.434861,49.213073,7.015203


In [7]:
import re
import pandas as pd
import matplotlib.pyplot as plt

def extract_dataset(model: str, group_name: str) -> str:
    """Return the dataset combo (e.g., 'outdoor_and_indoor', 'outdoor_indoor_and_garage')."""
    prefix = f"{group_name}_"
    if model.startswith(prefix) and "_run" in model:
        return model[len(prefix):].split("_run", 1)[0]
    # Fallback: take the chunk before `_run<digits>`, then strip a leading '<group>_'
    m = re.search(r"(.+?)_run\d+", model)
    if m:
        chunk = m.group(1)
        return chunk[len(prefix):] if chunk.startswith(prefix) else chunk
    return model  # very unlikely fallback


combined_results = {}

for group, rows in results.items():
    
    current_dataset_rows = []
    for current_row in rows:
        current_model_name = current_row[0]
        mae = current_row[1]
        mse = current_row[2]
        rmse = current_row[3]
        dataset = extract_dataset(current_model_name,group)

        current_dataset_rows.append([dataset,mae,mse,rmse])
    
    combined_results[group] = current_dataset_rows

    


import pandas as pd
for group, rows in combined_results.items():
    df = pd.DataFrame(rows, columns=["model", "mae", "mse", "rmse"])
    df = df.groupby("model")[["mae", "mse", "rmse"]].min().reset_index()
    df = df.rename(columns={
        "model": group,
        "mae": "MAE_avg",
        "mse": "MSE_avg",
        "rmse": "RMSE_avg"
    })

    # Style (gradient -> \cellcolor in LaTeX)
    styled = (
        df.style
          .hide(axis="index")
          .format(precision=2, decimal=",")      # PT decimal comma
          .background_gradient(cmap="RdYlGn_r")  # green=low, red=high
    )

    # Let Styler write LaTeX (keeps colors). Add a float env with caption/label.
    latex = styled.to_latex(
        caption=f"{group} – best (min) errors por modelo",
        label=f"tab:{group}_metrics",
        position="H",                # works with \usepackage{float}
        environment="table",         # wrap in \begin{table}...\end{table}
        convert_css=True,            # if your pandas supports it; safe to include
    )

    #display(styled)
    print(latex.replace("_","\_"))


\begin{table}[H]
\caption{CNN\_final\_meters – best (min) errors por modelo}
\label{tab:CNN\_final\_meters\_metrics}
\begin{tabular}{lrrr}
CNN\_final\_meters & MAE\_avg & MSE\_avg & RMSE\_avg \\
garage\_and\_indoor & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 7,27 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 88,01 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 9,38 \\
outdoor\_and\_garage & {\cellcolor[HTML]{DAF08D}} \color[HTML]{000000} 6,18 & {\cellcolor[HTML]{B1DE71}} \color[HTML]{000000} 61,78 & {\cellcolor[HTML]{C3E67D}} \color[HTML]{000000} 7,86 \\
outdoor\_and\_indoor & {\cellcolor[HTML]{E3F399}} \color[HTML]{000000} 6,22 & {\cellcolor[HTML]{F7FCB4}} \color[HTML]{000000} 67,81 & {\cellcolor[HTML]{FFFBB8}} \color[HTML]{000000} 8,23 \\
outdoor\_indoor\_and\_garage & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 5,43 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 49,21 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 7,02 \\
\end{tabular}
\end{table}



<>:66: SyntaxWarning: invalid escape sequence '\_'
<>:66: SyntaxWarning: invalid escape sequence '\_'
/tmp/ipykernel_3199718/1399987055.py:66: SyntaxWarning: invalid escape sequence '\_'
  print(latex.replace("_","\_"))
